<a href="https://colab.research.google.com/github/messias077/REP/blob/main/Problema_Generalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
""" Avaliando a generalização de algoritmos """

import numpy as np
import copy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, make_scorer, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine


""" Classe simplória para auxiliar na hora de rodar os modelos """
class Modelo():
    def __init__(self, modelo, descricao, aninhado=False):
        self.modelo = modelo
        self.descricao = descricao
        self.aninhado = aninhado  # Indica se fará o K-Fold aninhado
        self.acuracias = []  # Guarda a acurácia de cada rodada do modelo
    
    # Só para ficar mais simples na hora de chamar esses métodos
    def fit(self, X, y):
        self.modelo.fit(X, y)
    
    def predict(self, X):
        return self.modelo.predict(X)


# Carrega os dados
wine = load_wine()
X, y = wine.data, wine.target

def ajustar_modelo(X, y, qtd_folds, modelo):
    """ Faz ajuste de alguns hiperparâmetros dos modelos """
    # Gera as folds
    kf = KFold(n_splits=qtd_folds)

    # Dados para verificar o melhor modelo
    melhor_acuracia = 0
    melhor_modelo = cp.deepcopy(modelo)

    # Roda todas os n_splits do KFold
    for treinamento, teste in kf.split(X, y):
        X_treinamento = X[treinamento]
        y_treinamento = y[treinamento]

        X_teste = X[teste]
        y_teste = y[teste]

        # Sobrescreve o modelo que será afinado com o modelo padrão,
        # pois deverá refazer os ajustes dos hiperparâmetros via Grid Search
        modelo_ajustar = cp.deepcopy(modelo)

        modelo_ajustar.fit(X_treinamento, y_treinamento)
        y_pred = modelo_ajustar.predict(X_teste)
        acuracia = accuracy_score(y_teste, y_pred)

        if acuracia > melhor_acuracia:
            melhor_acuracia = acuracia
            melhor_modelo = modelo_ajustar

    return melhor_modelo

def rodar_modelos(modelos, qtd_folds, qtd_rodadas):
    """
        Roda as folds diversas vezes (dependendo dos parâmetros passados) para
        cada modelo e acumula as acurácias para tirar a média no final das
        execuções.
    """
    for _ in range(qtd_rodadas):
        # Gera as folds
        kf = KFold(n_splits=qtd_folds)

        # Roda todas os n_splits do KFold
        for treinamento, teste in kf.split(X, y):
            X_treinamento = X[treinamento]
            y_treinamento = y[treinamento]

            X_teste = X[teste]
            y_teste = y[teste]

            # Roda os modelos
            for mod in modelos:
                if mod.aninhado:
                    mod_aux = ajustar_modelo(X_treinamento, y_treinamento, qtd_folds-1, mod)
                else:
                    mod_aux = mod

                mod_aux.fit(X_treinamento, y_treinamento)
                y_pred = mod_aux.predict(X_teste)
                mod.acuracias.append(accuracy_score(y_teste, y_pred))

# Guarda os modelos que serão rodados
modelos = []

# Definição dos modelos

# Logistic Regression convencional
lr_conv = Pipeline([
    ("padronizacao", StandardScaler()),
    ("classificador", LogisticRegression())
])
modelos.append(Modelo(lr_conv, 'Logistic Regression - Convencional'))

# Logistic Regression aninhado
parametros_lr = {'max_iter': [1000, 3000, 5000]}
grid_search_lr = GridSearchCV(LogisticRegression(), parametros_lr, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
lr_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("classificador", grid_search_lr)
])
modelos.append(Modelo(lr_anin, 'Logistic Regression - Aninhado', True))

# KNN convencional
knn_conv = Pipeline([
    ("padronizacao", StandardScaler()),
    ("classificador", KNeighborsClassifier())
])
modelos.append(Modelo(knn_conv, 'KNeighbors Classifier - Convencional'))

# KNN Aninhado
parametros_knn = {'n_neighbors': [3, 5, 7]}
grid_search_knn = GridSearchCV(KNeighborsClassifier(), parametros_knn, scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
knn_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("gsknn", grid_search_knn)
])
modelos.append(Modelo(knn_anin, 'KNeighbors Classifier - Aninhado', True))

# Roda os modelos
rodar_modelos(modelos, 5, 10)

print("\n  --------------------------------------")
print("\n         **** Resultados ****")
print("\n  --------------------------------------")

for mod in modelos:
    print(f"\n * {mod.descricao}")
    print(f"   - Acurácia global..: {np.mean(mod.acuracias)}")
    print(f"   - Desvio padrão....: {np.std(mod.acuracias)}\n")



  --------------------------------------

         **** Resultados ****

  --------------------------------------

 * Logistic Regression - Convencional
   - Acurácia global..: 0.9609523809523809
   - Desvio padrão....: 0.028267341226138717


 * Logistic Regression - Aninhado
   - Acurácia global..: 0.9609523809523809
   - Desvio padrão....: 0.028267341226138717


 * KNeighbors Classifier - Convencional
   - Acurácia global..: 0.8934920634920632
   - Desvio padrão....: 0.06916410535377258


 * KNeighbors Classifier - Aninhado
   - Acurácia global..: 0.8992063492063492
   - Desvio padrão....: 0.07152726867264826

